In [1]:
pwd

'/content'

In [2]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
#folder = os.path.join('/content/drive/My Drive/`Capstone Data`')
os.chdir(r"drive/My Drive/Capstone Data")

In [4]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
df=pd.read_csv("MSME Data Modified_v1.9_featureEngineered.csv")

In [10]:
# Normalize the data
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer

In [7]:
!pip install catboost

     |████████████████████████████████| 66.3MB 55kB/s 


In [8]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import VotingClassifier
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

In [22]:
# get models
# get a voting ensemble of models
# define the base models
models = list()

decision_tree = Pipeline([('Scaler', RobustScaler()),('cart', DecisionTreeClassifier())])
models.append(('decision', decision_tree))

randomforest = Pipeline([('Scaler', RobustScaler()),('rf', RandomForestClassifier())])
models.append(('randomforest', randomforest))

svc = Pipeline([('Scaler', RobustScaler()),('svc', SVC())])
models.append(('svc', svc))

knn = Pipeline([('Scaler', RobustScaler()),('knn', KNeighborsClassifier())])
models.append(('knn', knn))

gnb = Pipeline([('Scaler', RobustScaler()),('nb', GaussianNB())])
models.append(('knn', knn))

cat = Pipeline([('Scaler', RobustScaler()),('cb', CatBoostClassifier(iterations=10))])
models.append(('cat', cat))

lgb = Pipeline([('Scaler', RobustScaler()),('lgbm', LGBMClassifier())])
models.append(('lgb', lgb))

xgb = Pipeline([('Scaler', RobustScaler()),('xgb',  XGBClassifier())])
models.append(('xgb', xgb))

# define the voting ensemble
ensemble = VotingClassifier(estimators=models, voting='hard')

In [23]:
models

[('decision', Pipeline(memory=None,
           steps=[('Scaler',
                   RobustScaler(copy=True, quantile_range=(25.0, 75.0),
                                with_centering=True, with_scaling=True)),
                  ('cart',
                   DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                          criterion='gini', max_depth=None,
                                          max_features=None, max_leaf_nodes=None,
                                          min_impurity_decrease=0.0,
                                          min_impurity_split=None,
                                          min_samples_leaf=1, min_samples_split=2,
                                          min_weight_fraction_leaf=0.0,
                                          presort='deprecated', random_state=None,
                                          splitter='best'))],
           verbose=False)), ('randomforest', Pipeline(memory=None,
           steps=[('Scale

In [24]:

ensemble

VotingClassifier(estimators=[('decision',
                              Pipeline(memory=None,
                                       steps=[('Scaler',
                                               RobustScaler(copy=True,
                                                            quantile_range=(25.0,
                                                                            75.0),
                                                            with_centering=True,
                                                            with_scaling=True)),
                                              ('cart',
                                               DecisionTreeClassifier(ccp_alpha=0.0,
                                                                      class_weight=None,
                                                                      criterion='gini',
                                                                      max_depth=None,
                                                

In [25]:
# Split the data into X & y

X = df.drop('loan_default', axis = 1).values
y = df['loan_default']

y = y.astype(int)

print(X.shape)
print(y.shape)

(32191, 48)
(32191,)


In [15]:
# Hold-out validation
from sklearn.model_selection import train_test_split
# first one
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, test_size=0.2, random_state=15)

# Second one
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, train_size = 0.9, test_size=0.1, random_state=15)

print(X_train.shape)
print(X_test.shape)
print(X_valid.shape)

print(y_train.shape)
print(y_test.shape)
print(y_valid.shape)

(23176, 48)
(6439, 48)
(2576, 48)
(23176,)
(6439,)
(2576,)


In [30]:
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings("ignore")

In [31]:
results = []
names = []
for name, model in models:
    kfold = KFold(n_splits=10, random_state=21)
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring='roc_auc')
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

decision: 0.521385 (0.014104)
randomforest: 0.614707 (0.015493)
svc: 0.528741 (0.036764)
knn: 0.550815 (0.017114)
knn: 0.550815 (0.017114)
Learning rate set to 0.5
0:	learn: 0.5547339	total: 67.3ms	remaining: 606ms
1:	learn: 0.5223394	total: 84.2ms	remaining: 337ms
2:	learn: 0.5115639	total: 101ms	remaining: 236ms
3:	learn: 0.5065892	total: 119ms	remaining: 178ms
4:	learn: 0.5035042	total: 136ms	remaining: 136ms
5:	learn: 0.5017186	total: 154ms	remaining: 102ms
6:	learn: 0.4998442	total: 171ms	remaining: 73.5ms
7:	learn: 0.4989428	total: 192ms	remaining: 47.9ms
8:	learn: 0.4976419	total: 209ms	remaining: 23.2ms
9:	learn: 0.4962595	total: 227ms	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.5552209	total: 16.5ms	remaining: 149ms
1:	learn: 0.5225091	total: 33.3ms	remaining: 133ms
2:	learn: 0.5123161	total: 50.1ms	remaining: 117ms
3:	learn: 0.5074397	total: 67.4ms	remaining: 101ms
4:	learn: 0.5048446	total: 85.4ms	remaining: 85.4ms
5:	learn: 0.5030425	total: 102ms	remaining: 68.3ms
6

In [34]:
cv = KFold(n_splits=10, random_state=21)
n_scores = cross_val_score(ensemble, X_train, y_train, scoring='roc_auc', cv=cv)

In [35]:
n_scores

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])

In [ ]:
import pickle
# open a file, where you ant to store the data
file = open('random_forest_clf_loan_default_model1.pkl', 'wb')

# dump information to that file
pickle.dump(rf_random, file)